In [1]:
import generative_model_score
score_model = generative_model_score.GenerativeModelScore()
score_model.lazy_mode(True)
import dataset
import TorchKmeans
train_loader = dataset.get_cifar1_dataset(2048, 32, shuffle=False)
data_cl_loader, numdata_in_cl = TorchKmeans.clusterset_with_gm_load_or_make(score_model, train_loader)
#score_model.load_or_make(train_loader)

import torch


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0


Files already downloaded and verified
use pickle file : %s ../inception_model_info/9aa4ddb3bd06dd3539a303459af7736c.pickle_cls_info.pickle
Using generated real image info.
Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ../dataset
    Split: Train
    StandardTransform
Transform: Compose(
               Resize(size=(32, 32), interpolation=bilinear)
               CenterCrop(size=(32, 32))
               ToTensor()
           )


In [5]:
device = 'cuda:0'
n_epoch = 1e+6

In [6]:
import torch.nn as nn
ndf = 64
ngf = 64
nz = 100
nc = 3
ngpu = 1

class Assigner(nn.Module):
    def __init__(self, ngpu, K):
        super(Assigner, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Flatten(),
            #  -- original code --
            # state size. (ndf*8) x 4 x 4
            #nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            #nn.Conv2d(ndf * 8, 1, 2, 1, 0, bias=False),
            # state size. 1x1x1
            #nn.Sigmoid()
            nn.Linear(2048, K)
        )

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)

        return output.view(-1, 1).squeeze(1)



In [7]:
assign = Assigner(ngpu, 100).to(device)
criterion = torch.nn.CrossEntropyLoss()
assign_optim = torch.optim.Adam(assign.parameters(), lr=1e-4)

In [8]:



epoch = 0
while epoch < n_epoch : 
    
    
    total_hit = 0
    for i, (data, cluster) in enumerate(data_cl_loader) :         
        batch_size = data.size(0)
        image_cuda = data.to(device)
        cluster_cuda = cluster.to(device)
        predict_cluster = assign(image_cuda).view(batch_size, -1)
        loss = criterion(predict_cluster, cluster_cuda)
        hit = torch.argmax(predict_cluster, dim=1) == cluster_cuda
        total_hit += torch.sum(hit).item()
        
        #if i >= 20 : break
        
        assign_optim.zero_grad()
        loss.backward()
        assign_optim.step()
        
    #acc = torch.sum(hit).float()/len(hit)
    acc = total_hit / len(data_cl_loader.dataset)
    print(epoch, acc, loss.item())
    epoch += 1
    
    if acc > 0.99 : break

0 0.0852 3.873690128326416
1 0.15288 3.54447340965271
2 0.19174 3.2875869274139404
3 0.22794 3.135127544403076
4 0.25988 2.935563564300537
5 0.29222 2.707251787185669
6 0.32648 2.6279666423797607
7 0.3599 2.4787747859954834
8 0.39228 2.474518299102783
9 0.42766 2.3468589782714844
10 0.46308 2.2009801864624023
11 0.49522 2.088731527328491
12 0.52792 1.9563382863998413
13 0.56532 1.85586678981781
14 0.5968 1.6894570589065552
15 0.62962 1.6122157573699951
16 0.66286 1.5023406744003296
17 0.69358 1.440284252166748
18 0.7251 1.33284592628479
19 0.75304 1.2711520195007324
20 0.78056 1.1403942108154297
21 0.80774 1.0958948135375977
22 0.83152 1.0542280673980713
23 0.85524 0.9318735599517822
24 0.87842 0.8323195576667786
25 0.895 0.8206472396850586
26 0.9127 0.7464110255241394
27 0.92506 0.662059485912323
28 0.939 0.5907319784164429
29 0.95022 0.5525522232055664
30 0.96198 0.5254458785057068
31 0.96892 0.46694254875183105
32 0.97674 0.43338772654533386
33 0.98136 0.372249573469162
34 0.98564 0

In [11]:
#torch.save(assign.state_dict(), 'cifar10_32_assin.model')
assign.load_state_dict(torch.load('cifar10_32_assin.model'))

<All keys matched successfully>

In [18]:
class ClusterDiscriminator(nn.Module):
    def __init__(self, n_class, n_hidden) : 
        self.n_class = n_class
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_class, n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden,1),
            nn.Sigmoid()
        )
    
    def forward(self, x) :
        x = self.net(x)
        return x

In [19]:
cd_net = ClusterDiscriminator(2048, 64).to(device)
criterion_bce= torch.nn.BCELoss()
cd_optim = torch.optim.Adam(cd_net.parameters(), lr=1e-4)

In [20]:

class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            #nn.ConvTranspose2d(     nz, ngf * 8, 4, 1, 0, bias=False),
            nn.ConvTranspose2d(     nz, ngf * 8, 2, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 2 x 2
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 4 x 4
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 8 x 8
            nn.ConvTranspose2d(ngf * 2,     ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 16 x 16
            nn.ConvTranspose2d(    ngf,      nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 32 x 32
        )

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)
        return output




In [21]:
netG = Generator(ngpu).to(device)
g_optim = torch.optim.Adam(netG.parameters(), lr=1e-4)

In [22]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            #nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Conv2d(ndf * 8, 1, 2, 1, 0, bias=False),
            # state size. 1x1x1
            nn.Sigmoid()
        )

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)

        return output.view(-1, 1)

In [23]:
netD = Discriminator(ngpu).to(device)
d_optim = torch.optim.Adam(netD.parameters(), lr=1e-4)

In [24]:
import wandb
wandb.init(project='cluster_gan', name='cluster')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hsyang1222 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.11.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [32]:
fixed_z = torch.randn(8*8, 100, 1, 1, device=device)
import torchvision.utils as vutils
def get_fixed_z_image_np(netG) : 
    netG.eval()
    fake_img = netG(fixed_z)
    fake_np = vutils.make_grid(fake_img.detach().cpu(), nrow=8).permute(1,2,0).numpy()
    netG.train()
    return fake_np
    

In [26]:
epoch = 0

In [33]:
assign.eval()
while epoch < 1e+6 : 
    for i, (data, cluster) in enumerate(data_cl_loader) :   
        batch_size = data.size(0)
        
        if batch_size != 2048 : continue
        
        image_cuda = data.to(device)
        
        ### train D by image
        # real image
        d_predict_real_image = netD(image_cuda)
        label_one = torch.ones(batch_size, 1, device=device)
        loss_d_real = criterion_bce(d_predict_real_image, label_one)
        
        # fake image
        fake_latent = torch.randn(batch_size, 100, 1, 1, device=device)
        fake_image = netG(fake_latent)
        d_predict_fake_image = netD(fake_image)
        label_zero = torch.zeros(batch_size, 1, device=device)
        loss_d_fake = criterion_bce(d_predict_fake_image, label_zero)
        
        # backward and step
        loss_d = loss_d_real + loss_d_fake
        netD.zero_grad()
        loss_d.backward()
        d_optim.step()
        
        
        ### train G by image
        # fake image
        fake_image = netG(fake_latent)
        d_predict_fake_image = netD(fake_image)
        loss_g_fake = criterion_bce(d_predict_fake_image, label_one)
        
        # backward and step
        loss_g = loss_g_fake
        netG.zero_grad()
        loss_g.backward()
        g_optim.step()
        
        ### train CD by assign cluster
        # real cluster
        real_assign = assign(image_cuda).view(batch_size, -1).detach()
        cd_predict_real_assign = cd_net(real_assign.T)
        label_one = torch.ones(100, 1, device=device)
        loss_cd_real = criterion_bce(cd_predict_real_assign, label_one)
        
        # fake cluster
        fake_assign = assign(fake_image).view(batch_size, -1).detach()
        cd_predict_fake_assign = cd_net(fake_assign.T)
        label_zero = torch.zeros(100, 1, device=device)
        loss_cd_fake = criterion_bce(cd_predict_fake_assign, label_zero)
        
        # backward and step
        loss_cd = loss_cd_real + loss_cd_fake
        cd_net.zero_grad()
        loss_cd.backward()
        cd_optim.step()
        
        
        ### train G by assign cluster
        # fake image
        fake_image = netG(fake_latent)
        fake_assign = assign(fake_image).view(batch_size, -1)
        cd_predict_fake_assign = cd_net(fake_assign.T)
        loss_gcd_fake = criterion_bce(cd_predict_fake_assign, label_one)
        
        #backward and step
        loss_gcd = loss_gcd_fake
        netG.zero_grad()
        loss_gcd.backward()
        g_optim.step()
        
        
   
    epoch += 1
    print(epoch, loss_g.item(), loss_d.item(), loss_cd.item(), loss_gcd.item())
    
    
    real_argmax = torch.argmax(real_assign, dim=1).float()
    fake_argmax = torch.argmax(fake_assign, dim=1).float()
    wandb.log({
            'step' : epoch,
            'real_assign' : real_argmax,
            'real_assign_mean' : torch.mean(real_argmax),
            'real_assign_std' : torch.std(real_argmax),
            'fake_assign' : fake_argmax,
            'fake_assign_mean' : torch.mean(fake_argmax),
            'fake_assign_std' : torch.std(fake_argmax),  
            'fake_img' : [wandb.Image(get_fixed_z_image_np(netG), caption='fixed z image')]
          })

4 3.7682042121887207 0.08562841266393661 0.958013117313385 1.244203805923462
5 4.4927825927734375 0.7206068634986877 1.0549236536026 1.1825389862060547
6 5.149367809295654 0.4505501091480255 1.1427099704742432 0.9391769170761108
7 3.6578173637390137 0.5375150442123413 1.178741455078125 0.9406687617301941


KeyboardInterrupt: 

In [52]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (3, 3)
def plt_cluster(real_assign, fake_assign) : 
    real_cluster = torch.argmax(real_assign, dim=1).cpu().detach().numpy()
    fake_cluster = torch.argmax(fake_assign, dim=1).cpu().detach().numpy()
    sns.kdeplot(real_cluster, label='real')
    sns.kdeplot(fake_cluster, label='fake')
    plt.legend()
    plt.show()

In [9]:
numdata_in_cl

tensor([476, 553, 349, 516, 436, 672, 352, 525, 483, 722, 577, 344, 290, 516,
        583, 670, 506, 610, 387, 459, 294, 540, 652, 547, 334, 757, 530, 538,
        537, 296, 490, 554, 441, 548, 356, 358, 740, 403, 437, 458, 414, 232,
        435, 404, 417, 442, 458, 588, 636, 552, 637, 498, 464, 453, 631, 507,
        520, 642, 217, 465, 384, 473, 699, 390, 646, 547, 314, 488, 529, 859,
        449, 487, 402, 596, 492, 543, 518, 421, 699, 284, 728, 450, 637, 376,
        434, 500, 527, 327, 486, 393, 361, 674, 523, 508, 449, 738, 532, 559,
        763, 367], device='cuda:0')

In [18]:
def rand_numdata_in_cl(real, device='cpu') : 
    numdata_in_cl = real
    rand_data = torch.randint(0, int(torch.mean(numdata_in_cl.float()))*2, size=numdata_in_cl.shape, device=device)
    return rand_data
    
predict_numdata_in_cl = rand_numdata_in_cl(numdata_in_cl, device=numdata_in_cl.device)

In [19]:
predict_numdata_in_cl

tensor([757, 674, 599,  92, 442, 205, 827, 687, 789, 743, 967, 456, 996, 217,
        945, 322, 212, 532, 559, 963, 959,  89, 708, 934, 509, 773,  17, 627,
        465, 296, 896, 968, 526, 386, 770, 745, 818, 331, 375, 172, 500, 693,
        277,  27, 905, 384, 455, 354,  41,  94, 971, 646, 409,  53, 606, 566,
        218, 310, 185, 675, 466, 696, 890, 257,  53, 322, 845, 257, 478,  46,
        514, 254, 448, 715, 129, 954, 767, 635, 392, 568, 383, 650, 423, 978,
        839, 262, 120, 611, 621, 694, 902, 311, 119, 823, 291, 369, 472, 806,
        804, 519], device='cuda:0')

In [44]:
score_each_cl = (numdata_in_cl - predict_numdata_in_cl) / torch.max(numdata_in_cl)

In [50]:
score_each_cl

tensor([-0.3271, -0.1409, -0.2910,  0.4936, -0.0070,  0.5437, -0.5530, -0.1886,
        -0.3562, -0.0244, -0.4540, -0.1304, -0.8219,  0.3481, -0.4214,  0.4051,
         0.3423,  0.0908, -0.2002, -0.5867, -0.7742,  0.5250, -0.0652, -0.4505,
        -0.2037, -0.0186,  0.5972, -0.1036,  0.0838,  0.0000, -0.4726, -0.4820,
        -0.0990,  0.1886, -0.4820, -0.4505, -0.0908,  0.0838,  0.0722,  0.3329,
        -0.1001, -0.5367,  0.1839,  0.4389, -0.5681,  0.0675,  0.0035,  0.2724,
         0.6927,  0.5332, -0.3888, -0.1723,  0.0640,  0.4657,  0.0291, -0.0687,
         0.3516,  0.3865,  0.0373, -0.2445, -0.0955, -0.2596, -0.2224,  0.1548,
         0.6903,  0.2619, -0.6182,  0.2689,  0.0594,  0.9464, -0.0757,  0.2712,
        -0.0536, -0.1385,  0.4226, -0.4785, -0.2899, -0.2491,  0.3574, -0.3306,
         0.4016, -0.2328,  0.2491, -0.7008, -0.4715,  0.2771,  0.4738, -0.3306,
        -0.1572, -0.3504, -0.6298,  0.4226,  0.4703, -0.3667,  0.1839,  0.4296,
         0.0698, -0.2875, -0.0477, -0.17

In [81]:
size = (1024,100)
while True : 
    rand_predict_cl = torch.rand(size, device='cuda:0')
    predict_label = predict_to_onehot(rand_predict_cl)
    win_cluster = torch.argmax(predict_label,dim=1)
    credit_calc = score_each_cl[win_cluster].view(-1,1)

    predict_credit = crediter(predict_label)
    loss = criterion_mse(predict_credit, credit_calc)
    crediter_optim.zero_grad()
    loss.backward()
    crediter_optim.step()
    
    print(loss.item())
    if loss.item() < 1e-2 : break
    

0.4581369161605835
0.4170662760734558
0.4550018906593323
0.4188308119773865
0.4347251355648041
0.4473688006401062
0.46169063448905945
0.46613115072250366
0.4633241295814514
0.43065738677978516
0.4343855381011963
0.45449888706207275
0.4264181852340698
0.4164388179779053
0.4751928746700287
0.4471169710159302
0.4421916604042053
0.4763880968093872
0.43872660398483276
0.4530807137489319
0.444631963968277
0.4345307946205139
0.44200679659843445
0.47160452604293823
0.44441574811935425
0.45058155059814453
0.44464367628097534
0.45227712392807007
0.46525585651397705
0.4197957515716553
0.4414604902267456
0.4438413977622986
0.429252564907074
0.4621813893318176
0.4355493187904358
0.4508194923400879
0.4456557333469391
0.43497535586357117
0.4404650330543518
0.46337801218032837
0.4546729624271393
0.41863173246383667
0.3939117193222046
0.4428812265396118
0.42026442289352417
0.44148099422454834
0.42824116349220276
0.44283327460289
0.4447316825389862
0.4298245310783386
0.4166997969150543
0.419566988945007

0.12622947990894318
0.10964471101760864
0.11597247421741486
0.11338827013969421
0.10124713182449341
0.1205313503742218
0.11722803115844727
0.11267293989658356
0.12135057151317596
0.11534678190946579
0.1051044762134552
0.11231175065040588
0.11013373732566833
0.1088929995894432
0.11627789586782455
0.10854437947273254
0.11635979264974594
0.11319077014923096
0.11182436347007751
0.11552713811397552
0.11769208312034607
0.12066302448511124
0.10316134244203568
0.1056741252541542
0.11016912013292313
0.11333512514829636
0.11111028492450714
0.10773174464702606
0.12126557528972626
0.11085756123065948
0.11393696069717407
0.10478472709655762
0.11494041234254837
0.11018645763397217
0.11075791716575623
0.11088752746582031
0.11156991124153137
0.10775317251682281
0.11451464891433716
0.11047294735908508
0.11067399382591248
0.1159309446811676
0.1012774333357811
0.11707453429698944
0.10540524125099182
0.12391029298305511
0.10623340308666229
0.11083175241947174
0.11693016439676285
0.11075575649738312
0.1093

0.08161857724189758
0.09116104245185852
0.07450123131275177
0.08244531601667404
0.0754084587097168
0.08796285837888718
0.08652974665164948
0.08354926109313965
0.08179929852485657
0.08056910336017609
0.08500716835260391
0.08006644248962402
0.08299651741981506
0.08278035372495651
0.08512986451387405
0.08077383786439896
0.07941918075084686
0.08483879268169403
0.07648338377475739
0.08310653269290924
0.08294972032308578
0.09237301349639893
0.08860330283641815
0.08102956414222717
0.08158720284700394
0.08038003742694855
0.08343158662319183
0.08124223351478577
0.08278690278530121
0.08939994871616364
0.07947829365730286
0.08170758187770844
0.075795017182827
0.08164286613464355
0.08269937336444855
0.0838814377784729
0.08518268167972565
0.08008766174316406
0.07926059514284134
0.08286521583795547
0.08014822751283646
0.08311520516872406
0.07409918308258057
0.07765711843967438
0.09313978254795074
0.08234798908233643
0.08117690682411194
0.08010442554950714
0.0810900330543518
0.08523307740688324
0.080

0.07491162419319153
0.07639185339212418
0.08364361524581909
0.07501496374607086
0.07886694371700287
0.0779109075665474
0.08112500607967377
0.08282819390296936
0.07167330384254456
0.0814443975687027
0.07471206784248352
0.07490553706884384
0.07855449616909027
0.07972632348537445
0.0777556300163269
0.0736335963010788
0.08676308393478394
0.08052050322294235
0.07441602647304535
0.08256281912326813
0.06768347322940826
0.07831104844808578
0.07447364181280136
0.07255204766988754
0.06215400621294975
0.08158192038536072
0.07680797576904297
0.07444076985120773
0.07934051752090454
0.08275043964385986
0.08388198912143707
0.07985886931419373
0.07942311465740204
0.08233936131000519
0.0754660964012146
0.08147337287664413
0.07834925502538681
0.07522320747375488
0.07979268580675125
0.07789841294288635
0.08164837956428528
0.07883914560079575
0.07940088212490082
0.0720178484916687
0.0742216408252716
0.07581787556409836
0.07369795441627502
0.0732765942811966
0.07878702878952026
0.0762663334608078
0.0813099

0.08021670579910278
0.07438255101442337
0.06682645529508591
0.07151535153388977
0.0735151395201683
0.07675155997276306
0.07797448337078094
0.07729938626289368
0.08216778934001923
0.08160725235939026
0.0815923810005188
0.07543154805898666
0.07824372500181198
0.08244543522596359
0.07049612700939178
0.07678012549877167
0.07812836766242981
0.07338730245828629
0.07987124472856522
0.08561288565397263
0.07623488456010818
0.0631050243973732
0.0720328837633133
0.06909802556037903
0.07498612254858017
0.07616614550352097
0.08103857934474945
0.07775069773197174
0.0824146568775177
0.0805455893278122
0.07569124549627304
0.07545094192028046
0.07814405858516693
0.07175033539533615
0.07574399560689926
0.07713349163532257
0.0821937620639801
0.07360817492008209
0.08231332898139954
0.07585272192955017
0.07773414254188538
0.07787143439054489
0.08005286008119583
0.07613793015480042
0.06961128115653992
0.06576088070869446
0.06523653864860535
0.07593580335378647
0.07167546451091766
0.0766739770770073
0.077442

0.07432456314563751
0.08027733117341995
0.07461966574192047
0.07824507355690002
0.07842150330543518
0.07169373333454132
0.07823820412158966
0.0759633332490921
0.06394332647323608
0.07270302623510361
0.0707181990146637
0.07303772121667862
0.0696285292506218
0.0788731575012207
0.07801894843578339
0.07736015319824219
0.070781409740448
0.07847511768341064
0.07542493939399719
0.07676580548286438
0.0772661343216896
0.07063363492488861
0.07392476499080658
0.0776917040348053
0.07526372373104095
0.07683435082435608
0.07097573578357697
0.08711349219083786
0.07244226336479187
0.07552268356084824
0.08327075839042664
0.07693839073181152
0.0736670047044754
0.07216598838567734
0.07626417279243469
0.07641805708408356
0.07936146855354309
0.07711689174175262
0.07732796669006348
0.07191658765077591
0.07039352506399155
0.08598610758781433
0.07786776125431061
0.07526931911706924
0.08153856545686722
0.07600262761116028
0.07332988828420639
0.07049843668937683
0.07951721549034119
0.07336324453353882
0.0754538

0.07863414287567139
0.07058729231357574
0.07149898260831833
0.07219024002552032
0.06872031837701797
0.07559545338153839
0.07286673784255981
0.07264962792396545
0.08376546204090118
0.07526910305023193
0.07776408642530441
0.07308101654052734
0.07184740900993347
0.07729646563529968
0.07201309502124786
0.07679952681064606
0.08428344130516052
0.07486361265182495
0.07569418847560883
0.07486949115991592
0.07679249346256256
0.08073092997074127
0.07035308331251144
0.07771681249141693
0.07322055101394653
0.07795075327157974
0.07405570149421692
0.07112668454647064
0.07733504474163055
0.07667869329452515
0.07473289966583252
0.07262416929006577
0.08129795640707016
0.07606226205825806
0.07254014164209366
0.06789736449718475
0.07466340810060501
0.07090163230895996
0.0696067065000534
0.07081489264965057
0.07876405119895935
0.07399804145097733
0.08383655548095703
0.07591100037097931
0.07559703290462494
0.07090356945991516
0.0683741569519043
0.0698583647608757
0.07165542244911194
0.07479878515005112
0.0

0.06233290210366249
0.07347869873046875
0.07807447761297226
0.07245762646198273
0.06574001163244247
0.07922204583883286
0.07211382687091827
0.0719943642616272
0.07470829784870148
0.07271599769592285
0.07675832509994507
0.07219187915325165
0.08525317162275314
0.07088637351989746
0.07010387629270554
0.07045207917690277
0.07659322023391724
0.07855123281478882
0.07586780190467834
0.06833265721797943
0.07006910443305969
0.07496529817581177
0.0666860044002533
0.06739559769630432
0.07112037390470505
0.07737390697002411
0.0744435042142868
0.07375387102365494
0.06901197135448456
0.08172497153282166
0.06829787790775299
0.06447038054466248
0.0736733227968216
0.06858782470226288
0.08084013313055038
0.07443232834339142
0.0703745037317276
0.0666457936167717
0.08039923757314682
0.07337270677089691
0.07750901579856873
0.07639166712760925
0.06376603245735168
0.074793741106987
0.07217293232679367
0.07034125924110413
0.07660934329032898
0.07700741291046143
0.0751834586262703
0.07585911452770233
0.0743691

0.07850974798202515
0.07563191652297974
0.08107037842273712
0.07329663634300232
0.08082932233810425
0.07955220341682434
0.07187232375144958
0.07329641282558441
0.07092569768428802
0.07212712615728378
0.07375604659318924
0.07297185808420181
0.07334019243717194
0.07615554332733154
0.08234474062919617
0.07124112546443939
0.06721968948841095
0.07566014677286148
0.0691489428281784
0.07164435088634491
0.07017432153224945
0.07428483664989471
0.08019126951694489
0.07987771183252335
0.07297886162996292
0.07562936842441559
0.08771906793117523
0.07992932200431824
0.07542064785957336
0.07642805576324463
0.06762272119522095
0.07270301133394241
0.08088992536067963
0.07882153242826462
0.07564488798379898
0.07980071008205414
0.0669698566198349


KeyboardInterrupt: 

In [71]:
def predict_to_onehot(predict, label_num=100) : 
    predict_label = torch.argmax(predict, dim=1)
    zeros = torch.zeros((len(predict_label), label_num), device=predict.device)
    zeros[torch.arange(len(zeros)),predict_label]=1
    return zeros

# todo

assigner 학습하고 고정

gan학습 진행하면서
assigner로 cluster를 찾음
fake image cluster를 만듬
credit net을 학습함
credit_score = credit(assinger(G(rand)))

maximize credit_score

In [ ]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()


    parser.add_argument('--device', type=str, default='cuda')
    parser.add_argument('--epochs', type=int, default=100)
    parser.add_argument('--batch_size', type=int, default=64)
    parser.add_argument('--img_size', type=int, default=32)
    parser.add_argument('--save_image_interval', type=int, default=5)
    parser.add_argument('--loss_calculation_interval', type=int, default=5)
    parser.add_argument('--latent_dim', type=int, default=10)
    parser.add_argument('--n_iter', type=int, default=3)
    parser.add_argument('--project_name', type=str, default='AAE')
    parser.add_argument('--dataset', type=str, default='', choices=['LSUN_dining_room', 'LSUN_classroom', 'LSUN_conference', 'LSUN_churches',
                                                                    'FFHQ', 'CelebA', 'cifar10', 'mnist', 'mnist_fashion', 'emnist'])

    parser.add_argument('--model_name', type=str, default='', choices=['vanilla', 'yeop_loss', 
                                                                       'yeop_n_iter', 'mod_var', 'mod2_var', 'gme', 'latent_mapping', 'gme_inference', 'direct'])

    parser.add_argument('--lr', type=float, default=1e-4)
    parser.add_argument('--run_test', type=bool, default=False)

    args = parser.parse_args()

    main(args)